In [1]:
from db.database import Database
from os import path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
from db.models import Post, Topic, Blueprint

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Adrian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
db = Database()
topics = db.get_topics()
posts = db.get_posts()
blueprints = db.get_all_blueprints()

Loading blueprints: 100%|██████████| 2196/2196 [00:01<00:00, 1442.40it/s]


In [ ]:
topic_1 = topics[10]
posts_topic_1 = db.get_posts_by_topic_id(topic_1.id)
print(posts_topic_1[0])

In [17]:
posts_topic_1[0].post_url

'https://community.home-assistant.io//t/nag-prompt-blueprint-android-notification/255041/1'